In [2]:
# !pip install torch numpy transformers datasets tiktoken wandb tqdm

In [3]:
# Loading training text data
with open("nanogpt/input.txt", "r", encoding='utf-8') as f:
    text = f.read()
print(f'Total number of characters = {len(text)}')
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'Total number of unique characters = {vocab_size}')
print(f"Characters = {''.join(chars)}")

Total number of characters = 1115394
Total number of unique characters = 65
Characters = 
 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [4]:
# simple tokenizer 1
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encoder1 = lambda x: [stoi[ch] for ch in x]
decoder1 = lambda x: ''.join([itos[i] for i in x])

In [5]:
print(encoder1('hello'))
print(decoder1(encoder1('hello')))

[46, 43, 50, 50, 53]
hello


In [6]:
# !pip install sentencepiece

In [8]:
# sentencepiece tokenizer 2
import sentencepiece as spm
params = ('--input=./nanogpt/input.txt ' '--model_prefix=spm ' '--vocab_size=1000 ')
spm.SentencePieceTrainer.Train(params)
sp = spm.SentencePieceProcessor()
sp.Load('spm.model')



True

In [22]:
print(sp.EncodeAsPieces('Hello world.'))
print(sp.EncodeAsIds('Hello world.'))
print(sp.DecodeIds([151, 88, 21, 887, 6]))

['▁He', 'll', 'o', '▁world', '.']
[184, 65, 25, 427, 7]
ter Toaaking:


In [10]:
# !pip install tiktoken

In [14]:
# tiktoken tokenizer 3
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")
enc.n_vocab

100277

In [15]:
print(enc.encode("hello world"))
print(enc.decode(enc.encode("hello world")))


[15339, 1917]
hello world


In [16]:
# tokenizer 1
print(encoder1('hello'))
print(decoder1(encoder1('hello')))

# tokenizer 2
print(sp.EncodeAsIds('Hello world.'))
print(sp.DecodeIds([151, 88, 21, 887, 6]))

# tokenizer 3
print(enc.encode("hello world"))
print(enc.decode(enc.encode("hello world")))

[46, 43, 50, 50, 53]
hello
[184, 65, 25, 427, 7]
ter Toaaking:
[15339, 1917]
hello world


In [22]:
import torch
data1 = torch.tensor(encoder1(text),dtype=torch.long)
data2 = torch.tensor(sp.EncodeAsIds(text),dtype=torch.long)
data3 = torch.tensor(enc.encode(text),dtype=torch.long)
print(f'Using simple tokenizer, there are {data1.shape[0]} tokens, \n Using sentencepiece tokenizer, there are {data2.shape[0]} tokens, \n Using tiktoken tokenizer, there are {data3.shape[0]} tokens')
print(f'Simple tokenizer: {data1[:50]} \n {decoder1(data1[:50].tolist())}')
print(f'Sentencepiece tokenizer: {data2[:50]} \n {sp.DecodeIds(data2[:50].tolist())}')
print(f'Tiktoken tokenizer: {data3[:50]} \n {enc.decode(data3[:50].tolist())}')

Using simple tokenizer, there are 1115394 tokens, 
 Using sentencepiece tokenizer, there are 407344 tokens, 
 Using tiktoken tokenizer, there are 301829 tokens
Simple tokenizer: tensor([18, 47, 56, 57, 58,  1, 15, 47, 58, 47, 64, 43, 52, 10,  0, 14, 43, 44,
        53, 56, 43,  1, 61, 43,  1, 54, 56, 53, 41, 43, 43, 42,  1, 39, 52, 63,
         1, 44, 59, 56, 58, 46, 43, 56,  6,  1, 46, 43, 39, 56]) 
 First Citizen:
Before we proceed any further, hear
Sentencepiece tokenizer: tensor([298, 537,   6, 259, 280,  12,  81, 231, 107,  35, 384, 987,   3, 223,
         33, 254,   7,  52,  65,   6,  78, 358,  21,  73,   3, 254,   7, 298,
        537,   6, 205, 106,  91, 115,   5, 201, 167,   8, 752,  14, 414, 216,
         14,  50, 190, 321,  26,  52,  65,   6]) 
 First Citizen: Before we proceed any further, hear me speak. All: Speak, speak. First Citizen: You are all resolved rather to die than to famish? All:
Tiktoken tokenizer: tensor([ 5451, 47317,   512, 10438,   584, 10570,   904,  4726,

In [ ]:
# train-test split
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
# one example of input-target pair
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f'context = {context}, target = {target}')
    print(f'        = {decoder(context.tolist())},        = {decoder([target.tolist()])}')

context = tensor([18]), target = 47
        = F,        = i
context = tensor([18, 47]), target = 56
        = Fi,        = r
context = tensor([18, 47, 56]), target = 57
        = Fir,        = s
context = tensor([18, 47, 56, 57]), target = 58
        = Firs,        = t
context = tensor([18, 47, 56, 57, 58]), target = 1
        = First,        =  
context = tensor([18, 47, 56, 57, 58,  1]), target = 15
        = First ,        = C
context = tensor([18, 47, 56, 57, 58,  1, 15]), target = 47
        = First C,        = i
context = tensor([18, 47, 56, 57, 58,  1, 15, 47]), target = 58
        = First Ci,        = t


In [ ]:
torch.manual_seed(1337)
batch_size = 4
block_size = 8

def get_batch(data, batch_size=batch_size, block_size=block_size):
    idx = torch.randint(0, data.size(0) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in idx])
    y = torch.stack([data[i+1:i+block_size+1] for i in idx])
    return x, y

In [ ]:
xb, yb = get_batch(train_data)
print('inputs:')
print(xb.shape)
print(xb) # our input to the transformer
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[47, 57, 10,  1, 39, 52, 42,  1],
        [59, 56,  1, 46, 43, 39, 56, 58],
        [32, 46, 39, 58,  1, 39, 50, 61],
        [26, 53, 58, 46, 47, 52, 45,  1]])
targets:
torch.Size([4, 8])
tensor([[57, 10,  1, 39, 52, 42,  1, 50],
        [56,  1, 46, 43, 39, 56, 58, 57],
        [46, 39, 58,  1, 39, 50, 61, 39],
        [53, 58, 46, 47, 52, 45,  1, 40]])
----
when input is [47] the target: 57
when input is [47, 57] the target: 10
when input is [47, 57, 10] the target: 1
when input is [47, 57, 10, 1] the target: 39
when input is [47, 57, 10, 1, 39] the target: 52
when input is [47, 57, 10, 1, 39, 52] the target: 42
when input is [47, 57, 10, 1, 39, 52, 42] the target: 1
when input is [47, 57, 10, 1, 39, 52, 42, 1] the target: 50
when input is [59] the target: 56
when input is [59, 56] the target: 1
when input is [59, 56, 1] the target: 46
when input is [59, 56, 1, 46] the target: 43
when input is [59, 56, 1, 46, 43] the target: 39
when input is [59, 5

In [ ]:
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        # this is the embedded representation of the input, where idx is the input xb tensor
        logits = self.token_embedding_table(idx) # (B,T,C), C = vocab_size = number of classes

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C) # reshape due to cross-entropy
            targets = targets.view(B*T) # reshape due to cross-entropy
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens): # to generate new tokens
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens): # max_new_tokens is the maximum number of tokens to generate
            # get the predictions
            logits, _ = self(idx) # loss is none here
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities for each class
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution using the probabilities
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

# example training sample (B, 1)
example_x = torch.zeros((batch_size, 1), dtype=torch.long)

print(decoder(m.generate(idx = example_x, max_new_tokens=100)[0].tolist()))


torch.Size([32, 65])
tensor(4.7489, grad_fn=<NllLossBackward0>)

Sr?qP-QWktXoL&jLDJgOLVz'RIoDqHdhsV&vLLxatjscMpwLERSPyao.qfzs$Ys$zF-w,;eEkzxjgCKFChs!iWW.ObzDnxA Ms$3


In [ ]:
optimizer = torch.optim.AdamW(m.parameters(), lr=0.001)

In [ ]:
batch_size = 32
block_size = 8

for steps in range(10000):
    xb, yb = get_batch(train_data, batch_size, block_size)

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
print(loss.item())

2.3493263721466064


In [ ]:
example_x = torch.zeros((batch_size, 1), dtype=torch.long)
print(decoder(m.generate(idx = example_x, max_new_tokens=100)[0].tolist()))


OUbit alavest lelisovewhakererertof q-arathothe Inthe cashit:
NAn sh wim
Y!
Wece s.
As hie, ous;
HY,
